In [18]:
import sqlite3 as sql
import pandas as pd
import torch
import numpy as np
import json
import matplotlib.pyplot as plt
import matplotlib
import dateutil
import datetime
import sklearn
import preprocessing

In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
S_data_2019_04_04 = preprocessing.Preprocessing('S_data_2019_04_04','/Users/cm/20190403/','home-assistant_v2.db')

In [32]:
rooms = ['schlafzimmer','bad','kuche','wohnzimmer']

In [33]:
S_data_2019_04_04.load_data(rooms=rooms)

In [36]:
S_data_2019_04_04.get('train_raw_climatewohnzimmer').head()

,entity_id,state,attributes,last_updated,current_temp,set_temp
0,climate.wohnzimmer_wandthermostat_wz,manual,"{""current_temperature"": 16.7, ""min_temp"": 4.5,...",2019-03-21 20:58:15.919782,16.7,16.5
1,climate.wohnzimmer_wandthermostat_wz,manual,"{""current_temperature"": 16.9, ""min_temp"": 4.5,...",2019-03-21 21:03:20.902970,16.9,16.5
2,climate.wohnzimmer_wandthermostat_wz,manual,"{""current_temperature"": 16.7, ""min_temp"": 4.5,...",2019-03-21 21:13:30.925355,16.7,16.5
3,climate.wohnzimmer_wandthermostat_wz,manual,"{""current_temperature"": 16.9, ""min_temp"": 4.5,...",2019-03-21 21:18:33.946381,16.9,16.5
4,climate.wohnzimmer_wandthermostat_wz,manual,"{""current_temperature"": 16.7, ""min_temp"": 4.5,...",2019-03-21 21:23:40.920109,16.7,16.5
